In [1]:
print("Hii Aski!!")

Hii Aski!!


In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt 
%matplotlib inline

/opt/anaconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
# Read in all the words
words = open('names.txt').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
# Build the vocabulary of characters and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i , s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s , i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict next character
X , Y = [] , []

for w in words:
    # print(w)

    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context) , '------->' , itos[ix])
        context = context[1:] + [ix] # crop and append


X = torch.tensor(X)
Y = torch.tensor(Y)

In [7]:
X.shape , X.dtype , Y.shape , Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [8]:
# built the dataset 

def built_dataset(words):
    block_size = 3 # context length: how many characters do we take to predict next character
    X , Y = [] , []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape , Y.shape)
    return X , Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr , Ytr = built_dataset(words[:n1])
Xdev , Ydev = built_dataset(words[n1:n2])
Xte , Yte = built_dataset(words[n2:])


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [9]:
C = torch.randn((27 , 2))

In [10]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [11]:
W1 = torch.randn((6 , 100))
b1 = torch.randn(100)

In [12]:
h = torch.tanh(emb.view(-1 , 6) @ W1 + b1)

In [13]:
h

tensor([[ 0.8586,  0.6585, -0.5129,  ...,  0.1838,  0.9167,  0.3313],
        [ 0.6614,  0.9895,  0.4535,  ..., -0.6464,  0.9970,  0.9664],
        [ 0.2634,  0.9093, -0.9919,  ...,  0.9458,  0.3637, -0.8396],
        ...,
        [-0.9486, -0.4861, -0.9508,  ..., -0.8601, -0.3999,  0.9605],
        [ 0.8277, -0.5405, -0.9461,  ..., -0.9878,  0.5545,  0.3414],
        [-0.9337,  0.9456, -0.1805,  ...,  0.9762,  0.3537,  0.9977]])

In [14]:
h.shape

torch.Size([228146, 100])

In [15]:
W2 = torch.randn((100 , 27))
b2 = torch.randn(27)

In [16]:
logits = h @ W2 + b2

In [17]:
logits.shape

torch.Size([228146, 27])

In [18]:
counts = logits.exp()


In [19]:
prob = counts / counts.sum(1 , keepdims=True)

In [20]:
prob.shape

torch.Size([228146, 27])

In [22]:
loss = -prob[torch.arange(32) , Y].log().mean()
loss

IndexError: shape mismatch: indexing tensors could not be broadcast together with shapes [32], [228146]

# --------------------- now made respactable ------------------

In [23]:
Xtr.shape , Ytr.shape # Dataset

(torch.Size([182625, 3]), torch.Size([182625]))

In [24]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27 , 2) , generator=g)
W1 = torch.randn((6 , 300) , generator=g)
b1 = torch.randn(300 , generator=g)
W2 = torch.randn((300 , 27) , generator=g)
b2 = torch.randn(27 , generator=g)
parameters = [C , W1 ,b1 , W2 , b2]

In [25]:
sum(p.nelement() for p in parameters) # number of parameters in total

10281

In [26]:
for p in parameters:
    p.requires_grad = True

In [32]:
lre = torch.linspace(-3 , 0 , 1000)
lrs = 10**lre


In [33]:
# lri = []
# lossi = []
for i in range(3000):
    # minibatch construct
    ix = torch.randint(0 , Xtr.shape[0] , (32 , ))
    
    # Forward Pass
    emb = C[Xtr[ix]] # [32 , 3 , 2] 
    h = torch.tanh(emb.view(-1 , 6) @ W1 + b1) # [32 , 100]
    logits = h @ W2 + b2 # [32 , 27]
    # counts = logits.exp()
    # prob = counts / counts.sum(1 , keepdims = True)
    # loss = -prob[torch.arange(32) , Y].log().mean()
    loss = F.cross_entropy(logits , Ytr[ix])
    # print(loss.item())

    # Backward Pass
    for p in parameters:
        p.grad = None

    loss.backward()

    # Update
    # lr = lrs[i]
    lr = 10**-2
    for p in parameters:
        p.data += -lr * p.grad
    
    # # Track Stats
    # lri.append(lr)
    # lossi.append(loss.item())

print(loss.item())

2.7676923274993896


In [34]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1 , 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits , Ydev)
loss

tensor(2.5929, grad_fn=<NllLossBackward0>)

In [35]:
plt.plot(lri , lossi)

NameError: name 'lri' is not defined